![MLU Logo](../data/MLU_Logo.png)

# <a name="0">Machine Learning Accelerator - Computer Vision - Lecture 3</a>


## Inference with Pre-trained ResNet Model

In this notebook, we use pre-trained ResNet with only a few lines of code.

1. <a href="#1">Downloading a Pretrained Model</a>      
2. <a href="#2">Preprocessing an Image</a>
3. <a href="#3">Using ResNet50 for Inference</a>
    

In [1]:
! pip install -q -r ../requirements.txt

ERROR: Cannot install autogluon-core==1.1.1 and scikit-learn>=1.5.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


Let's import the necessary libraries:

In [2]:
%matplotlib inline
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## 1. <a name="1">Downloading a Pretrained Model</a>
(<a href="#0">Go to top</a>)

We will use a ResNet 50 neural net trained on ImageNet dataset as our base model.

In [3]:
net = torchvision.models.resnet50(pretrained=True)
net.eval()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 233MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## 2. <a name="2">Pre-processing an Image</a>
(<a href="#0">Go to top</a>)

Next we read a sample image, and pre-process it with preset data transforms.

In [7]:
img_raw

TypeError: 'PngImageFile' object is not subscriptable

In [4]:
img_raw = Image.open('../data/catdog.png')
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
img = transform(img_raw).unsqueeze(0)
plt.imshow(img_raw)
plt.axis('off')
plt.show()

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

## 3. <a name="3">Inference Using ResNet50</a>
(<a href="#0">Go to top</a>)

Now let's generate the predictions from the pretrained ResNet50.

In [ ]:
with torch.no_grad():
    pred = net(img)
print(len(pred[0]))

To access all the existing classes of ImageNet, we can use the class names from the ImageNet dataset. Let's take a look at the first 5 classes!

In [ ]:
with open('../data/imagenet_classes.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]
print(classes[:5])

Now let's see how the model interprets our input test image!

In [ ]:
topK = 5
probs, indices = torch.topk(torch.nn.functional.softmax(pred, dim=1), k=topK)
print('The input picture is classified to be')
for i in range(topK):
    print(f'\t[{classes[indices[0][i]]}], with probability {probs[0][i]:.3f}.')

Feel free to download and try other ResNet versions (ResNet18, ResNet101, ResNet152, etc.) in your own experiment. What is more, try to finetune on other datasets to see if you can improve the model performance.